In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from prophet import Prophet


In [ ]:
master_clean = pd.read_csv(
    "../data/processed/aadhaar_master_monthly.csv"
)

master_clean.head()


### Why Total Updates?

Demographic updates capture migration and data correction behavior,
while biometric updates capture age-transition and identity refresh needs.

Combining both provides a realistic measure of Aadhaar service demand.


In [ ]:
master_clean['total_updates'] = (
    master_clean['total_demo_updates'] +
    master_clean['total_bio_updates']
)


In [ ]:
state_monthly = (
    master_clean
    .groupby(['state', 'month'], as_index=False)
    ['total_updates']
    .sum()
)


In [ ]:
focus_state = "Delhi"

state_ts = state_monthly[
    state_monthly['state'] == focus_state
]


In [ ]:
# Compare top 3 states instead of only one
top_states = (
    master_clean
    .groupby('state')['total_updates']
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index
)

top_states


In [ ]:
df_prophet = (
    state_ts[['month', 'total_updates']]
    .copy()
)

df_prophet['ds'] = pd.to_datetime(df_prophet['month'])
df_prophet['y'] = df_prophet['total_updates']

df_prophet = (
    df_prophet[['ds', 'y']]
    .sort_values('ds')
    .reset_index(drop=True)
)



In [ ]:
if (
    df_prophet.shape[0] < 6 or
    df_prophet['y'].sum() == 0 or
    df_prophet['y'].nunique() <= 1
):
    raise ValueError("Insufficient or non-informative data for forecasting.")


In [ ]:
df_prophet['ds'] = pd.to_datetime(df_prophet['ds'])

# DO NOT force monthly continuity
# DO NOT interpolate

df_p = df_prophet.sort_values('ds')


In [ ]:
model = Prophet(
    yearly_seasonality=False,
    weekly_seasonality=False,
    daily_seasonality=False,
    changepoint_prior_scale=0.05,  # LOW flexibility → no overfitting
    interval_width=0.95            # Wide uncertainty (honest)
)

model.fit(df_prophet)


In [ ]:
# Ensure sorted & clean time index
df_p = df_p.sort_values('ds')

last_date = df_p['ds'].max()
forecast_end = pd.Timestamp(year=2026, month=12, day=31)

# Number of months to forecast (robust & explicit)
months_ahead = (
    (forecast_end.to_period('M') - last_date.to_period('M')).n
)

# Guardrail: avoid negative or zero forecast horizon
months_ahead = max(months_ahead, 1)

future = model.make_future_dataframe(
    periods=months_ahead,
    freq='M',
    include_history=True
)

forecast = model.predict(future)


In [ ]:
df_p = df_p.sort_values('ds')

last_date = df_p['ds'].max()
forecast_end = pd.Timestamp(year=2026, month=12, day=31)

months_ahead = (
    (forecast_end.to_period('M') - last_date.to_period('M')).n
)

months_ahead = max(months_ahead, 1)

future = model.make_future_dataframe(
    periods=months_ahead,
    freq='M',
    include_history=True
)

forecast = model.predict(future)


In [ ]:
# Select only 2026 forecasts
forecast_2026 = forecast[forecast['ds'].dt.year == 2026]

plt.plot(
    forecast_2026['ds'],
    forecast_2026['yhat'] * 0.85,
    linestyle='--',
    label='Conservative Scenario'
)

plt.plot(
    forecast_2026['ds'],
    forecast_2026['yhat'],
    linestyle='-',
    label='Base Scenario'
)

plt.plot(
    forecast_2026['ds'],
    forecast_2026['yhat'] * 1.15,
    linestyle='--',
    label='Aggressive Scenario'
)


In [ ]:
fig = model.plot(forecast)

ax = fig.gca()
ax.set_title(f"2026 Demand Forecast (Trend-Only) – {focus_state}")
ax.set_xlabel("Month")
ax.set_ylabel("Expected Updates")


In [ ]:
last_hist_date = df_p['ds'].max()

ax.axvline(
    x=last_hist_date,
    color='black',
    linestyle='--',
    alpha=0.7,
    label='Forecast Start'
)


In [ ]:
plt.show()

In [ ]:
plt.figtext(
    0.5, -0.1,
    "⚠️ 2026 forecasts are trend-based projections from limited 2025 data; "
    "they represent direction, not precise demand.",
    ha="center",
    fontsize=9
)


In [ ]:
trend_strength = (
    forecast['yhat'].iloc[-1] - forecast['yhat'].iloc[0]
) / max(forecast['yhat'].iloc[0], 1)

trend_strength

trend_percent = trend_strength * 100

direction = "Upward 📈" if trend_strength > 0 else "Downward 📉"

print(f"Estimated Trend Direction for 2026: {direction}")
print(f"Approximate Trend Change: {trend_percent:.2f}%")
print("Note: This value is directional only and should not be interpreted as precise growth.")



In [ ]:
forecasts_summary = []

for state in top_states:
    ts = state_monthly[state_monthly['state'] == state].copy()

    # ---- State validation ----
    if ts.shape[0] < 6 or ts['total_updates'].sum() == 0 or ts['total_updates'].nunique() <= 1:
        forecasts_summary.append({
            "state": state,
            "status": "INSUFFICIENT_DATA",
            "base_forecast": None,
            "conservative": None,
            "aggressive": None
        })
        continue

    # ---- Prophet formatting ----
    df_p = ts[['month', 'total_updates']]
    df_p['ds'] = pd.to_datetime(df_p['month'])
    df_p['y'] = df_p['total_updates']
    df_p = df_p[['ds', 'y']].sort_values('ds')

    # ---- Trend-only Prophet ----
    m = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        changepoint_prior_scale=0.05,
        interval_width=0.95
    )
    m.fit(df_p)

    # ---- Forecast to end of 2026 ----
    last_date = df_p['ds'].max()
    forecast_end = pd.Timestamp(year=2026, month=12, day=31)

    months_ahead = max(
        (forecast_end.to_period('M') - last_date.to_period('M')).n, 1
    )

    future = m.make_future_dataframe(periods=months_ahead, freq='M')
    f = m.predict(future)

    f_2026 = f[f['ds'].dt.year == 2026]

    base = f_2026['yhat'].sum()

    forecasts_summary.append({
        "state": state,
        "status": "OK",
        "base_forecast": base,
        "conservative": base * 0.85,
        "aggressive": base * 1.15
    })

forecast_df = pd.DataFrame(forecasts_summary)
forecast_df


In [ ]:
forecast_df['uncertainty_ratio'] = (
    forecast_df['aggressive'] - forecast_df['base_forecast']
) / forecast_df['base_forecast']


In [ ]:
def risk_bucket(x):
    if x >= 0.30:
        return "🔴 High Uncertainty"
    elif x >= 0.15:
        return "🟡 Medium Uncertainty"
    else:
        return "🟢 Low Uncertainty"

forecast_df['risk_level'] = forecast_df['uncertainty_ratio'].apply(risk_bucket)

forecast_df.sort_values(by='uncertainty_ratio', ascending=False)


In [ ]:
forecast_df['risk_adjusted_demand'] = (
    forecast_df['base_forecast'] *
    (1 - forecast_df['uncertainty_ratio'])
)

q_low = forecast_df['risk_adjusted_demand'].quantile(0.33)
q_high = forecast_df['risk_adjusted_demand'].quantile(0.66)

def priority_bucket(x):
    if x >= q_high:
        return "🔴 High Priority"
    elif x >= q_low:
        return "🟡 Medium Priority"
    else:
        return "🟢 Low Priority"

forecast_df['priority_level'] = (
    forecast_df['risk_adjusted_demand']
    .apply(priority_bucket)
)

forecast_df
